<a href="https://colab.research.google.com/github/vothane/tripleplay/blob/master/tripleplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.preprocessing.image import load_img, img_to_array
from keras import backend as K
import numpy as np
import os.path

Using TensorFlow backend.


Run this cell only if you do not already have the image data.

***Recommend that you not run this on your own machine.***

In [27]:
!git clone https://github.com/vothane/tripleplay.git
!mv tripleplay/imgs imgs
!rm -rf tripleplay

Cloning into 'tripleplay'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 122 (delta 11), reused 105 (delta 3), pack-reused 0
Receiving objects: 100% (122/122), 12.02 MiB | 33.63 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [39]:
f = lambda img_file : img_to_array(load_img('imgs/{}'.format(img_file), color_mode = "grayscale"))
pitcher_map = {img_file[:-4] : f(img_file)
              for img_file in os.listdir('imgs/')}
print(np.shape(pitcher_map['Chris_Sale']))

(600, 600, 1)


In [50]:
image_size = 600 # x=y

# network parameters
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
latent_dim = 16
# encoder/decoder number of CNN layers and filters per layer
layer_filters = [32, 64]

# build the autoencoder model
# first build the encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = inputs

# stack of Conv2D(32)-Conv2D(64)
for filters in layer_filters:
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=2,
               activation='relu',
               padding='same')(x)

# shape info needed to build decoder model so we don't do hand computation
# the input to the decoder's first Conv2DTranspose will have this shape
shape = K.int_shape(x)

# generate the latent vector
x = Flatten()(x)
latent = Dense(latent_dim, name='latent_vector')(x)

# instantiate encoder model
encoder = Model(inputs, latent, name='encoder')
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 600, 600, 1)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 300, 300, 32)      320       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 150, 150, 64)      18496     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1440000)           0         
_________________________________________________________________
latent_vector (Dense)        (None, 16)                23040016  
Total params: 23,058,832
Trainable params: 23,058,832
Non-trainable params: 0
_________________________________________________________________


In [41]:
# build the decoder model
latent_inputs = Input(shape=(latent_dim,), name='decoder_input')
# use the shape (7, 7, 64) that was earlier saved
x = Dense(shape[1] * shape[2] * shape[3])(latent_inputs)
# from vector to suitable shape for transposed conv
x = Reshape((shape[1], shape[2], shape[3]))(x)

# stack of Conv2DTranspose(64)-Conv2DTranspose(32)
for filters in layer_filters[::-1]:
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides=2,
                        activation='relu',
                        padding='same')(x)

# reconstruct the denoised input
outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          padding='same',
                          activation='sigmoid',
                          name='decoder_output')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1440000)           24480000  
_________________________________________________________________
reshape_3 (Reshape)          (None, 150, 150, 64)      0         
_________________________________________________________________
conv2d_transpose_5 (Conv2DTr (None, 300, 300, 64)      36928     
_________________________________________________________________
conv2d_transpose_6 (Conv2DTr (None, 600, 600, 32)      18464     
_________________________________________________________________
decoder_output (Conv2DTransp (None, 600, 600, 1)       289       
Total params: 24,535,681
Trainable params: 24,535,681
Non-trainable params: 0
________________________________________________________________

In [42]:
# autoencoder = encoder + decoder
# instantiate autoencoder model
autoencoder = Model(inputs,
                    decoder(encoder(inputs)),
                    name='autoencoder')
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 600, 600, 1)       0         
_________________________________________________________________
encoder (Model)              (None, 16)                23058832  
_________________________________________________________________
decoder (Model)              (None, 600, 600, 1)       24535681  
Total params: 47,594,513
Trainable params: 47,594,513
Non-trainable params: 0
_________________________________________________________________


In [43]:
# Mean Square Error (MSE) loss function, Adam optimizer
autoencoder.compile(loss='mse', optimizer='adam')

train_data = np.array([data for _, data in pitcher_map.items()])

autoencoder.fit(train_data, train_data, epochs=15, batch_size=batch_size)

W0617 23:43:23.202967 139768558991232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0617 23:43:23.400922 139768558991232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/15
96/96 [==============================] - 71s 742ms/step - loss: 1378.3313
Epoch 2/15
96/96 [==============================] - 63s 653ms/step - loss: 1374.9674
Epoch 3/15
96/96 [==============================] - 63s 656ms/step - loss: 1374.9085
Epoch 4/15
96/96 [==============================] - 63s 656ms/step - loss: 1374.8645
Epoch 5/15
96/96 [==============================] - 63s 654ms/step - loss: 1374.8015
Epoch 6/15
96/96 [==============================] - 63s 655ms/step - loss: 1374.7188
Epoch 7/15
96/96 [==============================] - 63s 657ms/step - loss: 1374.6765
Epoch 8/15
96/96 [==============================] - 63s 656ms/step - loss: 1374.6433
Epoch 9/15
96/96 [==============================] - 63s 652ms/step - loss: 1374.5881
Epoch 10/15
96/96 [==============================] - 63s 653ms/step - loss: 1374.5647
Epoch 11/15
96/96 [==============================] - 63s 651ms/step - loss: 1374.5709
Epoch 12/15
96/96 [==============================] - 62s 649ms/

first pass at tsne

In [0]:
from sklearn.manifold import TSNE

deep_features = encoder.predict(train_data, batch_size=batch_size)

X_embedded = TSNE(n_components=2).fit_transform(deep_features)

really this is basically nearest neighbors where closet points are pitchers who closely resemble each other. I need to
graph these points and label them with pitchers names to make sense of it all. after that we can ascertain what these 
deep features may be, right now there are 16 and we can use ensemble algos to find an optimal number after words. i'm guessing something like arm slot (3/4 or overhand) and left/right handedness.

In [60]:
X_embedded

array([[ 2.1230102 , -2.8864439 ],
       [ 3.015935  , -1.361749  ],
       [-2.7690341 , -1.937283  ],
       [ 2.627432  ,  1.2357011 ],
       [-0.9062499 ,  3.567927  ],
       [ 0.2653146 , -4.990888  ],
       [-0.36850187, -2.3039494 ],
       [ 2.9421334 , -5.188261  ],
       [ 2.7214847 ,  0.45871472],
       [-1.0072621 , -1.4214925 ],
       [ 0.08863518,  3.9763267 ],
       [-1.6808023 , -3.511221  ],
       [ 1.0835378 , -4.3475184 ],
       [-0.5858672 ,  1.7613571 ],
       [ 1.2900186 , -0.33795804],
       [ 3.9725788 , -3.5741136 ],
       [-1.6186585 , -0.5814358 ],
       [ 2.1624408 , -0.5831266 ],
       [ 0.31183073, -2.4945543 ],
       [ 0.40849626, -2.264806  ],
       [ 1.4047298 ,  4.097754  ],
       [ 0.24590746,  0.9957252 ],
       [-0.5598806 , -4.4652705 ],
       [-1.2933031 ,  2.5710597 ],
       [ 0.11562246,  2.3999634 ],
       [-0.57684064, -0.1192752 ],
       [ 0.4941725 ,  0.01633733],
       [-0.6489046 ,  3.812134  ],
       [ 3.9903512 ,